In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List

load_dotenv(override=True)

True

In [2]:

from my_custom_tools.arXivTool import ArXivTool
from my_custom_tools.DownloadTool import DownloadPaperTool

tool = ArXivTool()
results = tool.run(None, topic="Cat", max_results=1)

for paper in results:
    print(paper["title"])
    print(paper["link"])
    print()

download_tool = DownloadPaperTool()
message = download_tool.run(None, papers=results)
print(message)

Semistability and CAT(0) Geometry
http://arxiv.org/abs/1703.07003v1

http://arxiv.org/abs/1703.07003v1
✅ Downloaded 1 paper into the 'papers' folder
